- Inaya Rahmanisa
- 2106708330

For Programming Assignment 3, you are asked to perform re-ranking with learning-to-rank model on the given dataset. In order to do so, you need to preprocess the dataset, index the document collection, ideate the re-ranking features, train the model, and evaluate the performance for further analysis.

PyTerrier is a helpful tool to carry out IR operations. Therefore, we provide a PyTerrier template for this assignment. More information and example can be referred from the previous [Tutorial 1](https://colab.research.google.com/drive/10NpHgtotDQv6bQXEv3CmqyRVQ0BXknlC?usp=sharing) and PyTerrier documentation.


---


*   Fill in the `✋TODO` section
*   Provide justification for your choices
*   Report your experiment results and analysis


---


Happy exploring!

# Setup

In [ ]:
!pip install python-terrier -q
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git -q
!pip install datasets

You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [308]:
!pip show python-terrier

Name: python-terrier
Version: 0.11.0
Summary: Terrier IR platform Python API
Home-page: https://github.com/terrier-org/pyterrier
Author: Craig Macdonald
Author-email: craigm@dcs.gla.ac.uk
License: UNKNOWN
Location: c:\users\inaya\appdata\local\programs\python\python310\lib\site-packages
Requires: dill, tqdm, statsmodels, more-itertools, requests, ir-datasets, pytrec-eval-terrier, wget, pyjnius, deprecated, ir-measures, numpy, jinja2, matchpy, chest, pandas, joblib, scipy
Required-by: pyterrier-t5


In [2]:
import re
import gdown
import pyterrier as pt
import pandas as pd
import numpy as np

from datasets import load_dataset
from pyterrier.measures import *

if not pt.started():
  pt.init(version='snapshot')

c:\Users\inaya\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\inaya\AppData\Local\Temp\ipykernel_2912\3070504568.py:10: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: jitpack 2024-09-24 17:43), helper_version=0.0.8]
C:\Users\inaya\AppData\Local\Temp\ipykernel_2912\3070504568.py:11: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
The following code will have the same effect:
pt.terrier.set_version('snapshot')
pt.java.init() # optional, forces java initialisation
  pt.init(version='snapshot')


# Data Preparation

In [78]:

dataset = pt.get_dataset('irds:cord19/trec-covid/round1')

itera = (dataset.get_corpus_iter())
# for i in itera:
#     print(i)

cord19/trec-covid/round1 documents:   0%|          | 0/51078 [52:21<?, ?it/s]


In [28]:
itera = (dataset.get_corpus_iter())
for i in itera:
    print(i.keys())
    break
# print(itera)

cord19/trec-covid/round1 documents:   0%|          | 0/51078 [00:48<?, ?it/s]

dict_keys(['title', 'doi', 'date', 'abstract', 'docno'])


In [144]:
collections = pd.DataFrame(dataset.get_corpus_iter())
queries = pd.DataFrame(dataset.get_topics())

qrels_df = pd.DataFrame(dataset.get_qrels())

cord19/trec-covid/round1 documents: 100%|██████████| 51078/51078 [00:00<00:00, 86594.55it/s]

There are multiple query fields available: ('title', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


In [145]:
final_qrels = qrels_df.drop(columns="iteration", axis=1)
duplicate_count = final_qrels.duplicated(subset=["qid", "docno"]).sum()

print(f"Number of duplicated qid-docno pairs: {duplicate_count}")


Number of duplicated qid-docno pairs: 0


In [178]:
from sklearn.model_selection import train_test_split
unique_qids = final_qrels["qid"].unique()


# Split qids into train, dev, and test sets
train_qids, temp_qids = train_test_split(unique_qids, test_size=0.4, random_state=42)
dev_qids, test_qids = train_test_split(temp_qids, test_size=0.5, random_state=42)

# Filter qrels based on splits


train_qrels = final_qrels[final_qrels["qid"].isin(train_qids)]
dev_qrels = final_qrels[final_qrels["qid"].isin(dev_qids)]
test_qrels = final_qrels[final_qrels["qid"].isin(test_qids)]

# Print sizes of each split
print(f"Train set: {len(train_qrels)} rows")
print(f"Dev set: {len(dev_qrels)} rows")
print(f"Test set: {len(test_qrels)} rows")

Train set: 5365 rows
Dev set: 1674 rows
Test set: 1652 rows


In [147]:
collections.head()

,title,doi,date,abstract,docno
0,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,2003-01-13,"BACKGROUND: Rhinovirus, the most common cause ...",xqhn0vbp
1,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,2003-04-28,Recent analyses of human pathogens have reveal...,gi6uaa83
2,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,2003-06-27,"The army of the men of death, in John Bunyan's...",le0ogx1s
3,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,2003-09-12,BACKGROUND: The human leukocyte antigen (HLA) ...,fy4w7xz8
4,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,2003-09-10,BACKGROUND: An epidemic of a Severe Acute Resp...,0qaoam29


In [148]:
queries.head()

,qid,title,description,narrative
0,1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


In [149]:
train_qrels.head()

,qid,docno,label
323,2,01yc7lzk,0
324,2,084o1dmp,0
325,2,0eu2ikwa,0
326,2,0gxxuyln,0
327,2,0m5umout,0


## ✋ TODO

In [150]:
import unicodedata
import re

def remove_nonalphanum(text):
  pattern = re.compile('[\W_]+')
  return pattern.sub(' ', text)

def normalize_to_english(text): 
    text = text.lower()

    # normalize to decompose characters (e.g., "â" -> "â")
    normalized_text = unicodedata.normalize('NFD', text)
    # remove diacritic marks using regex (e.g., "â" -> "a")
    normalized_text = re.sub(r'[\u0300-\u036f]', '', normalized_text)
    return normalized_text

In [151]:
# TODO: preprocess data according to needs

collections['title'] = collections['title'].apply(lambda x: normalize_to_english(remove_nonalphanum(x)))
collections['abstract'] = collections['abstract'].apply(lambda x: normalize_to_english(remove_nonalphanum(x)))
collections.drop_duplicates(inplace=True)
collections = collections.drop_duplicates(subset=["docno"], keep="last")


collections = collections.rename(columns={'title': 'text'})

queries['description'] = queries['description'].apply(lambda x: normalize_to_english(remove_nonalphanum(x)))
queries.drop(columns=["title", "narrative"], inplace=True)

In [152]:
# preprocess queries
queries = queries.sort_values(by='qid').reset_index(drop=True)
queries

,qid,description
0,1,what is the origin of covid 19
1,10,has social distancing had an impact on slowing...
2,11,what are the guidelines for triaging patients ...
3,12,what are best practices in hospitals and at ho...
4,13,what are the transmission routes of coronavirus
5,14,what evidence is there related to covid 19 sup...
6,15,how long can the coronavirus live outside the ...
7,16,how long does coronavirus remain stable on sur...
8,17,are there any clinical trials available for th...
9,18,what are the best masks for preventing infecti...


In [153]:
# preprocess train_qrels
train_qrels = train_qrels.sort_values(by='qid').reset_index(drop=True)

# preprocess dev_qrels
dev_qrels = dev_qrels.sort_values(by='qid').reset_index(drop=True)

# preprocess test_qrels
test_qrels = test_qrels.sort_values(by='qid').reset_index(drop=True)

Mengecek distribusi label pada qrels

In [287]:
import pandas as pd

# Function to down-sample the majority class
def downsample_majority_class(df, label_column='label'):
    # Find the minimum number of samples in the non-majority classes
    min_class_size = df[label_column].value_counts().min()
    
    # Down-sample the majority class to match the smallest class size
    df_majority = df[df[label_column] == 0]
    df_minority = df[df[label_column] != 0]
    
    # Randomly sample from the majority class to match the smallest class size
    df_majority_downsampled = df_majority.sample(min_class_size, random_state=42)
    
    # Combine the downsampled majority class with the minority classes
    balanced_df = pd.concat([df_majority_downsampled, df_minority])
    
    return balanced_df

# Apply down-sampling on train, dev, and test sets
train_qrels_balanced = downsample_majority_class(train_qrels)
dev_qrels_balanced = downsample_majority_class(dev_qrels)
test_qrels_balanced = downsample_majority_class(test_qrels)

# Check the value counts after balancing
print(train_qrels_balanced['label'].value_counts())
print(dev_qrels_balanced['label'].value_counts())
print(test_qrels_balanced['label'].value_counts())


label
2    714
0    651
1    651
Name: count, dtype: int64
label
1    229
0    224
2    224
Name: count, dtype: int64
label
2    299
0    235
1    235
Name: count, dtype: int64


In [290]:
print(train_qrels_balanced['label'].value_counts())
print(dev_qrels_balanced['label'].value_counts())
print(test_qrels_balanced['label'].value_counts())

label
2    714
0    651
1    651
Name: count, dtype: int64
label
1    229
0    224
2    224
Name: count, dtype: int64
label
2    299
0    235
1    235
Name: count, dtype: int64


In [291]:
# train_topics = queries[queries['qid'].isin(train_qrels['qid'])]
# dev_topics = queries[queries['qid'].isin(dev_qrels['qid'])]
# test_topics = queries[queries['qid'].isin(test_qrels['qid'])]

train_topics = queries[queries['qid'].isin(train_qrels_balanced['qid'])]
dev_topics = queries[queries['qid'].isin(dev_qrels_balanced['qid'])]
test_topics = queries[queries['qid'].isin(test_qrels_balanced['qid'])]


#  get the first x qids (queries)
# def first_x_qids(topic_set, max_query):
#     first_100_qids = topic_set.drop_duplicates(subset='qid').head(max_query)
#     serp_qrels_first_100_qids = topic_set[topic_set['qid'].isin(first_100_qids['qid'])]
#     return serp_qrels_first_100_qids


# # get first 50 qids for topics and qrels
# train_topics = first_x_qids(train_topics, 50)
# dev_topics = first_x_qids(dev_topics, 50)
# test_topics = first_x_qids(test_topics, 50)
# test_topics = test_topics.rename(columns={'text': 'query'})
# dev_topics = dev_topics.rename(columns={'text': 'query'})
# train_topics = train_topics.rename(columns={'text': 'query'})


# train_qrels= train_qrels[:50]
# dev_qrels = dev_qrels[:50]
# test_qrels = test_qrels[:50]

In [169]:
# train_qrels= train_qrels[:50]
# dev_qrels = dev_qrels[:50]
# test_qrels = test_qrels[:50]

In [189]:
train_topics.shape
# dev_topics.shape
# test_topics.shape

(18, 2)

In [157]:
collections.head()

,text,doi,date,abstract,docno
0,airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,2003-01-13,background rhinovirus the most common cause of...,xqhn0vbp
1,discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,2003-04-28,recent analyses of human pathogens have reveal...,gi6uaa83
2,a new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,2003-06-27,the army of the men of death in john bunyan s ...,le0ogx1s
3,association of hla class i with severe acute r...,10.1186/1471-2350-4-9,2003-09-12,background the human leukocyte antigen hla sys...,fy4w7xz8
4,a double epidemic model for the sars propagation,10.1186/1471-2334-3-19,2003-09-10,background an epidemic of a severe acute respi...,0qaoam29


In [159]:

queries.head()

,qid,description
0,1,what is the origin of covid 19
1,10,has social distancing had an impact on slowing...
2,11,what are the guidelines for triaging patients ...
3,12,what are best practices in hospitals and at ho...
4,13,what are the transmission routes of coronavirus


In [182]:
train_qrels.head()

,qid,docno,label
323,2,01yc7lzk,0
324,2,084o1dmp,0
325,2,0eu2ikwa,0
326,2,0gxxuyln,0
327,2,0m5umout,0


# Indexing

## ✋ TODO

In [162]:
# TODO: adjust indexer
import os
import shutil


index_path = os.path.join(os.getcwd(), "index")  # Use current working directory + index


if os.path.exists(index_path):
    # If it exists, remove the directory and all its contents
    shutil.rmtree(index_path)

# index_path = "./index"
pd_indexer = pt.IterDictIndexer(index_path, \
                          type = pt.index.IndexingType(1), \
                          tokeniser = pt.index.TerrierTokeniser('utf'), \
                          stemmer = pt.index.TerrierStemmer('porter'), \
                          stopwords = pt.index.TerrierStopwords('terrier'), \
                          blocks = True, \
                          verbose = True,
                         meta={'docno': 20, 'text':4096 , 'doi':4096, 'date':1080, 'abstract': 10800})
index_ref = pd_indexer.index(collections.to_dict(orient="records"))



12:41:02.753 [main] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (8is9x9sc) - further warnings are suppressed
12:41:19.138 [main] WARN org.terrier.structures.indexing.Indexer -- Indexed 168 empty documents


In [163]:
index_fact = pt.IndexFactory.of(index_ref)
print(index_fact.getCollectionStatistics().toString())
print(index_fact.getMetaIndex().getKeys())

Number of documents: 51045
Number of terms: 25000
Number of postings: 473366
Number of fields: 1
Number of tokens: 484664
Field names: [text]
Positions:   true

['docno', 'text', 'doi', 'date', 'abstract']


In [164]:
print(index_fact.getMetaIndex().getItem("docno", 10))
# lihat lexicon
for i, (term, le) in enumerate(index_fact.getLexicon()):
  if i % 300 == 0:
    # limit output
    print(term, le.getFrequency())

5s6acr7m
0 104
1910 1
30th 3
654 1
abram 1
aefi 1
alphacoronaviru 26
annulata 2
arau 1
atopi 6
bafilomycin 2
bet 1
bl 1
bronz 1
camouflag 3
ccl22 1
chemiluminesc 8
clandestin 1
commerci 104
cordatum 1
crystallograph 22
daphnia 1
deploi 7
dimension 45
dp 2
edicion 1
endocrinologist 1
eras 1
expans 49
fern 1
focal 7
g15p 1
germani 53
gp350 2
haemagglutinin 14
hemolymphat 1
hmbpp 2
hydrocarbon 8
iga 24
inconsist 1
intercurr 2
isoflavon 3
kaa 1
l1 12
lentivir 28
longev 7
mainten 27
mdea 1
mhc 65
mlb1 1
mucolipin 1
nairoviru 6
neu5gc 1
noncontact 2
nunavut 1
operophtera 1
oxidas 7
paramyxovirina 1
pepitop 1
photoantimicrobi 1
pluripot 9
potenzi 1
procyanidin 3
publishedin 1
radiographi 9
reg 3
revista 1
rotterdam 1
sarcopt 1
sen 1
siderocalin 1
soil 5
stai 17
subtilas 1
t10b9 1
tetramethylen 1
tobramycin 2
triangl 1
ube3a 1
uptak 31
vergleich 1
vp13 1
woefulli 1
zorgelijk 1


In [165]:
di = index_fact.getDirectIndex()
doi = index_fact.getDocumentIndex()
lex = index_fact.getLexicon()

for posting in di.getPostings(doi.getDocumentEntry(0)):
    termid = posting.getId()
    lee = lex.getLexiconEntry(termid)
    print(f"Term ID: {termid}, Term: {lee.getKey()}, Frequency: {posting.getFrequency()}")

Term ID: 0, Term: rhinoviru, Frequency: 1
Term ID: 1, Term: detect, Frequency: 2
Term ID: 2, Term: irradi, Frequency: 1
Term ID: 3, Term: airborn, Frequency: 1
Term ID: 4, Term: ultraviolet, Frequency: 1
Term ID: 5, Term: assai, Frequency: 1
Term ID: 6, Term: effect, Frequency: 1
Term ID: 7, Term: rt, Frequency: 1
Term ID: 8, Term: semi, Frequency: 1
Term ID: 9, Term: pcr, Frequency: 1
Term ID: 10, Term: nest, Frequency: 1


# Baseline

In [166]:
bm25 = pt.BatchRetrieve(index_ref, wmodel = "BM25")

C:\Users\inaya\AppData\Local\Temp\ipykernel_2912\3225921946.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index_ref, wmodel = "BM25")


In [292]:
test_topics = test_topics.rename(columns={'description': 'query'})
dev_topics = dev_topics.rename(columns={'description': 'query'})
train_topics = train_topics.rename(columns={'description': 'query'})

In [293]:
eval_baseline = pt.Experiment(
    [bm25], # models
    test_topics,
    test_qrels_balanced,
    eval_metrics=[P@1, P@5, P@10, "map", nDCG@10],
    names=["BM25"], # for model's name readability
)
eval_baseline


,name,P@1,P@5,P@10,map,nDCG@10
0,BM25,0.833333,0.733333,0.55,0.176064,0.535262


In [193]:
# eval_baseline = pt.Experiment(
#     [bm25], # models
#     test_topics,
#     test_qrels, # qrels
#     eval_metrics=[P@1, P@5, P@10, "map", nDCG@10],
#     names=["BM25"], # for model's name readability
# )
# eval_baseline


,name,P@1,P@5,P@10,map,nDCG@10
0,BM25,0.833333,0.733333,0.55,0.176064,0.535262


In [194]:
serp = bm25.transform(test_topics)
serp

,qid,docid,docno,rank,score,query
0,1,29980,gv1k7u7j,0,19.805728,what is the origin of covid 19
1,1,44319,1mjaycee,1,17.981557,what is the origin of covid 19
2,1,30921,yn8nzezq,2,16.465071,what is the origin of covid 19
3,1,33536,h2uc7ria,3,15.798867,what is the origin of covid 19
4,1,48128,msohf5oa,4,15.184478,what is the origin of covid 19
...,...,...,...,...,...,...
5995,5,18414,v0zcbdhk,995,7.464044,what drugs have been active against sars cov o...
5996,5,18275,sojsk9y7,996,7.422614,what drugs have been active against sars cov o...
5997,5,42388,kp7qk49p,997,7.422614,what drugs have been active against sars cov o...
5998,5,45327,u9wmzvkz,998,7.422614,what drugs have been active against sars cov o...


# Re-rank

## Feature

### ✋ TODO

In [195]:
# TODO: propose and implement features

# Example:
def jaccard(doc, query):
    doc_set = set(doc.split())
    query_set = set(query.split())
    
    intersection = len(doc_set & query_set)
    union = len(doc_set | query_set)
    
    return intersection / union if union != 0 else 0.0

def other_feature(doc, query): # cosine similarity
    doc_tokens = doc.split()
    query_tokens = query.split()
    
    vocabulary = list(set(doc_tokens + query_tokens))
    
    doc_vector = np.array([doc_tokens.count(word) for word in vocabulary])
    query_vector = np.array([query_tokens.count(word) for word in vocabulary])
    
    dot_product = np.dot(doc_vector, query_vector)
    
    doc_magnitude = np.sqrt(np.dot(doc_vector, doc_vector))
    query_magnitude = np.sqrt(np.dot(query_vector, query_vector))
    
    if doc_magnitude == 0 or query_magnitude == 0:
        return 0.0  # handle edge case where one of the vectors has zero magnitude
    cosine_similarity = dot_product / (doc_magnitude * query_magnitude)
    
    return cosine_similarity

def generate_features(doc, query):
    return np.array([jaccard(doc, query), \
                     other_feature(doc, query)])

In [196]:
features = pt.apply.doc_features(lambda row: generate_features(row["text"], row["query"]))

In [197]:
K = 30

In [198]:
# Get "text" column using `pt.text.get_text(index_ref, "text")`
pipeline = (bm25 % K) >> pt.text.get_text(index_ref, "text") >> (features ** bm25)

## Learning-to-Rank

### ✋ TODO

In [31]:
!pip install xgboost-cpu



  Using cached xgboost_cpu-2.1.2-py3-none-win_amd64.whl (1.9 MB)


You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [199]:
# TODO: perform hyperparameter tuning on the LambdaMART
import xgboost as xgb

# Below hyperparameters are taken from the example on PyTerrier documentation:
# https://pyterrier.readthedocs.io/en/latest/ltr.html#gradient-boosted-trees-lambdamart

# XGBoost hyperparameters documentation:
# https://xgboost.readthedocs.io/en/stable/parameter.html

lmart_model = xgb.sklearn.XGBRanker(objective = 'rank:ndcg',
                              learning_rate = 0.1,
                              gamma = 1.0,
                              min_child_weight = 0.1,
                              max_depth = 6,
                              verbose = 2,
                              random_state = 42)

Hyperparameter Tuning

In [200]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'gamma': [0, 0.5, 1.0],
    'min_child_weight': [0.1, 0.5, 1.0],
    'max_depth': [3, 6, 9],
    'subsample': [0.5, 0.7, 0.8],
}

In [294]:
import itertools
from tqdm import tqdm

# Flatten the parameter grid into a list of dictionaries
param_combinations = [
    dict(zip(param_grid.keys(), values))
    for values in itertools.product(*param_grid.values())
]

best_score = -float('inf')  # init to negative infinity
best_params = None

for params in tqdm(param_combinations, desc="Hyperparameter Tuning"):
    
    lmart_model = xgb.sklearn.XGBRanker(objective='rank:ndcg', random_state=42, **params)
    
   
    lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_model, form="ltr")
    lmart_pipe.fit(train_topics, train_qrels_balanced, dev_topics, dev_qrels_balanced)
    # lmart_pipe.fit(train_topics, train_qrels, dev_topics, dev_qrels)


    results = lmart_pipe(dev_topics)
    score = pt.Utils.evaluate(results, dev_qrels_balanced,  metrics=['ndcg'])['ndcg']

    # score = pt.Utils.evaluate(results, dev_qrels, metrics=['ndcg'])['ndcg']

    if score > best_score:
        best_score = score
        best_params = params

print(f"Best score: {best_score}")
print(f"Best hyperparameters: {best_params}")






C:\Users\inaya\AppData\Local\Temp\ipykernel_2912\492794354.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  score = pt.Utils.evaluate(results, dev_qrels_balanced,  metrics=['ndcg'])['ndcg']




C:\Users\inaya\AppData\Local\Temp\ipykernel_2912\492794354.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  score = pt.Utils.evaluate(results, dev_qrels_balanced,  metrics=['ndcg'])['ndcg']




C:\Users\inaya\AppData\Local\Temp\ipykernel_2912\492794354.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  score = pt.Utils.evaluate(results, dev_qrels_balanced,  metrics=['ndcg'])['ndcg']




C:\Users\inaya\AppData\Local\Temp\ipykernel_2912\492794354.py:24: DeprecationWarning: Call to deprecated function (or staticmetho

Best score: 0.16728927834970073
Best hyperparameters: {'learning_rate': 0.05, 'gamma': 0.5, 'min_child_weight': 1.0, 'max_depth': 3, 'subsample': 0.5}


# Evaluation


## ✋ TODO

In [202]:
lmart_model = xgb.sklearn.XGBRanker(objective = 'rank:ndcg',    
                              verbose = 2,
                              random_state = 42,
                              **best_params)

In [295]:
lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_model, form = "ltr")
lmart_pipe.fit(train_topics, train_qrels_balanced, dev_topics, dev_qrels_balanced)

In [ ]:
# lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_model, form = "ltr")
# lmart_pipe.fit(train_topics, train_qrels, dev_topics, dev_qrels)

c:\Users\inaya\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [13:24:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-02e25fbcaf14bc601-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


In [296]:
# evaluate on dev set
dev_pred = lmart_pipe.transform(dev_topics)

eval_results = pt.Utils.evaluate(
    dev_pred,     
    dev_qrels_balanced   
)

print(eval_results)


{'map': 0.04075017155575187, 'ndcg': 0.151036477387153}


C:\Users\inaya\AppData\Local\Temp\ipykernel_2912\694728764.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  eval_results = pt.Utils.evaluate(


In [297]:
eval_results = pt.Experiment([ bm25, lmart_pipe], \
                            test_topics, \
                            test_qrels, \
                            eval_metrics=[P@10, "map", "recip_rank", nDCG@10], \
                            names=["BM25", "L2R"], \
                            baseline=0) # statistical significance test

In [298]:
eval_results

,name,map,recip_rank,P@10,nDCG@10,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P@10 +,P@10 -,P@10 p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,BM25,0.176064,0.888889,0.550000,0.535262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L2R,0.116940,0.861111,0.433333,0.413519,0.0,6.0,0.018864,1.0,1.0,0.891981,0.0,3.0,0.084242,2.0,4.0,0.23783


In [207]:
!pip install -U sentence-transformers -q

You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [208]:
from sentence_transformers import SentenceTransformer, CrossEncoder
from sentence_transformers.util import cos_sim

In [238]:
# TODO: Instantiate bi-encoder
# Feel free to choose the model
# A few examples of the pretrained model: https://sbert.net/docs/sentence_transformer/pretrained_models.html#original-models
bimodel = SentenceTransformer('all-MiniLM-L6-v2')

def _biencoder_apply(dataframe):
    # TODO: Encode the raw document and raw query using sentence transformer,
    # then compute their similarity score
    # Example: https://pyterrier.readthedocs.io/en/latest/text.html#examples-of-sentence-transformers
    # return 0
    query_embs = bimodel.encode(dataframe['query'].values)
    concatenated_text = dataframe['text'] + " " + dataframe['abstract']
    
    doc_embs = bimodel.encode(concatenated_text.values)
    
    
    scores = cos_sim(query_embs, doc_embs)
    return scores[0]

bi_encT = pt.apply.doc_score(_biencoder_apply, batch_size=128)

In [245]:
collections

,text,doi,date,abstract,docno
0,airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,2003-01-13,background rhinovirus the most common cause of...,xqhn0vbp
1,discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,2003-04-28,recent analyses of human pathogens have reveal...,gi6uaa83
2,a new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,2003-06-27,the army of the men of death in john bunyan s ...,le0ogx1s
3,association of hla class i with severe acute r...,10.1186/1471-2350-4-9,2003-09-12,background the human leukocyte antigen hla sys...,fy4w7xz8
4,a double epidemic model for the sars propagation,10.1186/1471-2334-3-19,2003-09-10,background an epidemic of a severe acute respi...,0qaoam29
...,...,...,...,...,...
51073,prevalence of antibodies against transmissible...,10.1016/j.vas.2018.11.003,2019-06-30,abstract transmissible gastroenteritis tge is ...,4xhcamks
51074,can we increase public awareness without creat...,10.1016/j.pec.2013.10.023,2014-02-28,,z2rqz3jx
51075,fast degrading polyesters as sirna nano carrie...,10.1016/j.jconrel.2008.06.010,2008-12-18,abstract a potential sirna carrier for pulmona...,wncuc903
51076,eif3f a central regulator of the antagonism at...,10.1016/j.biocel.2013.06.001,2013-10-31,abstract the eukaryotic initiation factor 3 su...,4x1ckr2i


In [240]:
CUT_OFF = 10

# bm25_biencoder = (bm25 % CUT_OFF) >> bi_encT
# bm25_biencoder = (bm25 % CUT_OFF) >> pt.text.get_text(index_ref,"text", "abstract") >> bi_encT


In [241]:
# TODO: Instantiate cross-encoder
# Feel free to choose the model
# A few examples of the pretrained model: https://sbert.net/docs/cross_encoder/pretrained_models.html
crossmodel = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)

def _crossencoder_apply(dataframe):
    # TODO: Use cross-encoder to directly predict the similarity score of the raw document and raw query
    # Example: https://pyterrier.readthedocs.io/en/latest/text.html#examples-of-sentence-transformers
    concatenated_text = dataframe['text'] + " " + dataframe['abstract']
    return crossmodel.predict(list(zip(dataframe['query'].values, concatenated_text.values)))

cross_encT = pt.apply.doc_score(_crossencoder_apply, batch_size=128)

In [242]:
# bm25_crossencoder = (bm25 % CUT_OFF) >> cross_encT


In [243]:
biencoder_crossencoder = pt.terrier.Retriever(index_ref,metadata=["docno", "text", "abstract"]) >> (bi_encT % CUT_OFF) >> cross_encT


In [248]:

# TODO: Perform experiment for every model
# (BM25, BM25 >> Bi-encoder, BM25 >> Cross-encoder, and Bi-encoder >> Cross-encoder),
# with significance testing, where the baseline is BM25
# Example: https://pyterrier.readthedocs.io/en/latest/experiments.html#significance-testing
eval_metrics=[P@10, "map", "recip_rank", nDCG@10]
eval_results = pt.Experiment([ bm25, biencoder_crossencoder, lmart_pipe],\
                           test_topics, \
                            test_qrels, \
                            eval_metrics=eval_metrics, \
                            names=["BM25",  "Biencoder CrossEncoder", "L2R"], \
                             filter_by_qrels=True,\
                            baseline=0) # statistical significance test































pt.apply: 100%|██████████| 6/6 [02:22<00:00, 23.81s/row]








pt.apply: 100%|██████████| 1/1 [00:06<00:00,  6.73s/row]


In [250]:
eval_results

,name,map,recip_rank,P@10,nDCG@10,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P@10 +,P@10 -,P@10 p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,BM25,0.176064,0.888889,0.550000,0.535262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Biencoder CrossEncoder,0.088633,1.000000,0.516667,0.576763,0.0,6.0,0.048915,1.0,0.0,0.363217,2.0,4.0,0.708934,3.0,3.0,0.658999
2,L2R,0.117786,0.783333,0.450000,0.409725,0.0,6.0,0.029809,1.0,1.0,0.489336,1.0,3.0,0.295559,1.0,5.0,0.270139


In [311]:
import pickle



def savemodels(model, model_filename):
    # Save models and results
    with open(f"{model_filename}.pkl", "wb") as f:
        # Save a dictionary containing models and eval_results
        pickle.dump(
             model
        , f)

savemodels(biencoder_crossencoder, 'biencoder_crossencoder')

In [305]:

lmart_pipe
with open('model.pkl','wb') as f:
    pickle.dump(bm25, f, protocol=pickle.HIGHEST_PROTOCOL)

In [312]:
savemodels(bm25, "bm25")

In [ ]:
savemodels(lmart_pipe, 'lmart_pipe')

In [302]:
savemodels(bm25, 'bm25')

In [315]:
query = "covid-19"

query = normalize_to_english(remove_nonalphanum(query))
queries_df = pd.DataFrame([{"qid": "1", "query": query}])

In [307]:
!pip show pyterrier


In [310]:
biencoder_crossencoder.transform(queries_df).sort_values(by=['rank'], ascending=[ True])

pt.apply: 100%|██████████| 1/1 [00:00<00:00,  1.08row/s]


,qid,docid,docno,text,abstract,score,query,rank
0,1,12,ri5v6u4x,towards evidence based gis driven national spa...,the term geographic information systems gis ha...,7.162597,gis riven health information infrastructure an...,0
2,1,24142,izporsu2,gis in health and human services,the chapter begins with a general overview of ...,0.178231,gis riven health information infrastructure an...,1
15,1,104,5dx3lscp,web gis in practice iii creating a simple inte...,this eye opener article aims at introducing th...,-0.083518,gis riven health information infrastructure an...,2
73,1,45727,102gvj2l,uk steps up its global health security,,-2.874480,gis riven health information infrastructure an...,3
426,1,20514,9xapri51,a multi data source surveillance system to det...,in 18 weeks health protection scotland hps dep...,-2.952474,gis riven health information infrastructure an...,4
136,1,24352,repm1vw9,public health surveillance a vital alert and r...,ungchusak heymann and pollack address the crit...,-4.567834,gis riven health information infrastructure an...,5
108,1,14938,d33ll0ka,strengthening health security at the hajj mass...,background hajj is one of the largest and the ...,-4.655541,gis riven health information infrastructure an...,6
68,1,3387,91uj6sph,steps to a sustainable public health surveilla...,more than a decade into the 21 st century the ...,-5.296201,gis riven health information infrastructure an...,7
515,1,1550,zhry4h3a,the global emerging infection surveillance and...,the armed forces health surveillance center gl...,-6.248644,gis riven health information infrastructure an...,8
116,1,19443,85ea18k8,a public health enhanced surveillance system f...,background from june 7 to june 9 2018 a g7 sum...,-6.542835,gis riven health information infrastructure an...,9


In [272]:
collections.iloc[12]

text        towards evidence based gis driven national spa...
doi                                     10.1186/1476-072x-3-1
date                                               2004-01-28
abstract    the term geographic information systems gis ha...
docno                                                ri5v6u4x
Name: 12, dtype: object

In [316]:
bm25.transform(queries_df).sort_values(by=['rank'], ascending=[ True])[:50]

,qid,docid,docno,rank,score,query
0,1,26284,5l7516xp,0,12.415991,covid 19
1,1,29800,8m3wg1tg,1,12.415991,covid 19
2,1,29872,450z0tv1,2,12.415991,covid 19
3,1,29902,s7idehep,3,12.415991,covid 19
4,1,30314,e5ya4vio,4,12.415991,covid 19
5,1,30331,j1q59u9c,5,12.415991,covid 19
6,1,32248,3gbxbg5f,6,12.415991,covid 19
7,1,34339,w11wop27,7,12.415991,covid 19
8,1,34406,6duc06et,8,12.415991,covid 19
9,1,34433,e4hmo4yc,9,12.415991,covid 19


In [286]:
# bm25.transform(queries_df)
lmart_pipe.transform(queries_df).sort_values(by=['rank'], ascending=[ True])

,qid,docid,docno,score,query,text,features,rank
0,1,12,ri5v6u4x,0.648330,gis riven health information infrastructure an...,towards evidence based gis driven national spa...,"[0.47368421052631576, 0.6581451817144176, 37.7...",0
1,1,24142,izporsu2,0.648330,gis riven health information infrastructure an...,gis in health and human services,"[0.4166666666666667, 0.6154574548966638, 31.54...",1
2,1,38028,452tu4ss,0.294833,gis riven health information infrastructure an...,redesign proposed for uk infrastructure to fig...,"[0.11764705882352941, 0.21320071635561041, 23....",2
3,1,45526,o6tuulli,0.204869,gis riven health information infrastructure an...,an audit of the use of isolation facilities in...,"[0.19047619047619047, 0.29250896965085227, 23....",3
4,1,24698,drjfwcdg,-0.218816,gis riven health information infrastructure an...,building the infrastructure and regulations ne...,"[0.23529411764705882, 0.41812100500354543, 17....",4
8,1,39442,1ru3wgkk,-0.435902,gis riven health information infrastructure an...,analysis of health stories in daily newspapers...,"[0.25, 0.4351941398892446, 16.04718364946167]",5
5,1,39228,rqjifww7,-0.554504,gis riven health information infrastructure an...,a consensus action agenda for achieving the na...,"[0.2222222222222222, 0.36363636363636365, 17.8...",6
6,1,50795,cugntk6v,-0.600260,gis riven health information infrastructure an...,tibetan health care takes back seat to infrast...,"[0.11764705882352941, 0.21320071635561041, 17....",7
13,1,18252,2bo8z52j,-0.785166,gis riven health information infrastructure an...,improving health services for african migrants...,"[0.15789473684210525, 0.3223291856101521, 15.2...",8
29,1,37135,ttzsv3kw,-0.861899,gis riven health information infrastructure an...,structure function analysis of the ge gi compl...,"[0.06666666666666667, 0.09196009797434058, 13....",9
